In [2]:
import pymupdf
from ollama import pull, generate
import math
import glob
from kokoro import KPipeline
from IPython.display import display, Audio
import torch

In [7]:
def process_pdf(input_path: str) -> list[str]:
    try:
        with pymupdf.open(input_path) as pdf:
            text_by_page = [page.get_text() for page in pdf]

    except Exception as e:
        raise Exception(f"PDF file processing failed. Please ensure that the file path is correct and that the file is not corrupted. Full Error: {e}")
    
    return text_by_page

def chunk_input(input: list, n_chunks: int = 5) -> list[list]:
    chunked_inputs = []

    for idx in range(math.ceil(len(input) / n_chunks)):
        start, end = idx * n_chunks, (idx+1) * n_chunks
        chunked_inputs.append(input[start:end])

    return chunked_inputs